### Authentication
First of all we authenticate in ee

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1ARtbsJoDdZzElAtxRos6DmdL0mbn8ATTaAhZxffbRziYiy_WC9fW9kb7pvc

Successfully saved authorization token.


In [54]:
# Get the desired zone shapefile
totoral = ee.FeatureCollection(
    '/home/aarja/Downloads/georeferences-main/shapefiles/totoral_las_lomitas/whole_farm/whole.geojson'
)

In [55]:
def clipper(image):
    global totoral
    return image.clip(totoral)

In [56]:
# Import the Sentinel-2 collection
sentinel_2_msi = ee.ImageCollection("COPERNICUS/S2_SR").filterDate('2022-10-01', '2022-10-17').map(clipper)

In [57]:
def calculate_ndvi(image):
    ndvi = image.expression(
        '(NIR - RED) / (NIR + RED)',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4')
        }
    )
    return ndvi

In [58]:
images = sentinel_2_msi.map(calculate_ndvi)

In [63]:
import folium
import json

In [60]:
lat, lon = -30.749311, -64.057867

my_map = folium.Map(location=[lat, lon], zoom_start=10)
my_map

In [61]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [62]:
# Set visualization parameters for land cover.
vis_params = {
    'min': -1,'max': 1,
    'palette': ['05450a','086a10', '54a708', '78d203', '009900', 'c6b044',
                'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44',
                'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4', '1c0dff']
}

my_map.add_ee_layer(images.first(), vis_params, 'NDVI')

EEException: Collection.loadTable: Collection asset '/home/aarja/Downloads/georeferences-main/shapefiles/totoral_las_lomitas/whole_farm/whole.geojson' not found.